In [1]:
import numpy as np
import csv
import time

np.random.seed(1234)
def randomize():
    np.random.seed(time.time())

In [8]:
RND_MEAN = 0
RND_STD = 0.0030
# 정규분포 난숫값의 평균과 표준편차로, 가중치 파라미터를 초기화 할 때 이용
LEARNING_RATE = 0.001

In [6]:
def abakibe_exec(epoch_count = 10, mb_size = 10, report = 1):
    load_abalone_dataset() #데이터셋을 읽어옴
    init_model() #모델의 파라미터들 초기화
    train_and_test(epoch_count, mb_size, report) #학습 및 평가 과정 수행

### 데이터 적재 함수 정의

In [7]:
def load_abalone_dataset():
    with open('data/chap01/abalone.csv') as file:
        csvreader = csv.reader(file)
        next(csvreader, None) #첫 행을 읽지 않고 건너뜀
        rows = []
        for row in csvreader:
            rows.append(row)
            
        global data, input_cnt, output_cnt #전역 변수로 선언하여 다른 함수에서도 이용 가능
        input_cnt, output_cnt = 10, 1 #입출력 벡터 크기
        data = np.zeros([len(rows), input_cnt + output_cnt])
        
        for n, row in enumerate(rows):
            if row[0] == 'I' : data[m, 0] = 1 #OneHotEncoding
            if row[0] == 'M' : data[m, 1] = 1
            if row[0] == 'F' : data[m, 2] = 1
            data[n, 3:] = row[1:]

### 파라미터 초기화 함수 정의

In [11]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt]) #가중치 행렬 값들을 정규분포를 갖는 난숫값으로 초기화
    # 파라미터의 초깃값을 실행할 때마다 달라지게 만듦
    bias = np.zeros([output_cnt]) #초기에 지나친 영향을 주지 않도록 0으로 초기화

- weight 가중치 행렬의 크기는 [10, 1], bias 편향 벡터 [1]

### 학습 및 평가 함수 정의

In [12]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size) #데이터를 섞고 학습용과 평가용 데이터셋을 분리하는 역할
    # return값은 step_count 하나지만 함수 내에서 global 선언을 통한 데이터셋 분리 기능
    test_x, test_y = get_test_data() #evaluation, test에서 같은 데이터 사용을 위해 미리 선언
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count): #step count 값 만큼 mini-batch 처리
            train_x, train_y = get_train_data(mb_size, n) #mini batch data를 얻어와 run_train 함수로 학습
            loss, acc = run_train(train_x, train_y) #mini batch 단위의 정확도와 loss값 return
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0: #report = 보고 주기(acc, loss값 출력)
            acc = run_test(test_x, test_y)
            print('Epoch {} : loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'.format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc= run_test(test_x, test_y) #전체 epoch가 끝나면 최종 평가 진행
    print('\nFinal Test : final accuracy = {:5.3f}'.format(final_acc))